# Content Collection and Processing for TSE-NER Long-tail Entity Extraction

Here we will try to follow the full process required to collect and prepare the data required before we can train and use a NER model. In the second pipeline, we review the process of Term and Sentence Expansion, training, and finally using the Stanford NER Tagger.

In [1]:
# This is in case you update modules while working
%load_ext autoreload 
%autoreload 2

## Content Collection (a.k.a. getting a *lot* of papers)

For this step we will be using [sci-paper-miner](https://github.com/ronentk/sci-paper-miner) to download scientific publications from arXiv. Get the code from their repository, and use the following command to run it:

`python crawl_core.py <your-api>` 

In the `crawl_core` file, you can modify the topics and range of years that you want to download, for example here we selected papers only from 2017 for Artificial Intelligence, Computational Complexity, Cryptography and Security, Human-Computer Interaction, Logic in Computer Science, Mathematical Software, Multiagent Systems, Neural and Evolutionary Computing and Sound (in the `crawl_core` file they have all the topics, you can choose what to keep).  In `configs` you can write a name for the folder where the data will be stored.

In [2]:
import json
import os

In [2]:
# This is the path where all the json files were downloaded 

path = 'sci_paper_miner/data/arxiv_2006-2017_cs/raw_query/'

We know that the files we downloaded are in JSON format, but let's check their structure.
For this we can use `os.walk`.

In [3]:
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.json'):
            print(filename)

144_Computer Science - Artificial Intelligence_2017_5_0.json
144_Computer Science - Artificial Intelligence_2017_5_1.json
144_Computer Science - Artificial Intelligence_2017_5_2.json
144_Computer Science - Artificial Intelligence_2017_5_3.json
144_Computer Science - Artificial Intelligence_2017_5_4.json
144_Computer Science - Artificial Intelligence_2017_5_5.json
144_Computer Science - Computational Complexity_2017_4_0.json
144_Computer Science - Computational Complexity_2017_4_1.json
144_Computer Science - Computational Complexity_2017_4_2.json
144_Computer Science - Computational Complexity_2017_4_3.json
144_Computer Science - Computational Complexity_2017_4_4.json
144_Computer Science - Computational Complexity_2017_4_5.json
144_Computer Science - Computational Complexity_2017_4_6.json
144_Computer Science - Cryptography and Security_2017_8_0.json
144_Computer Science - Cryptography and Security_2017_8_1.json
144_Computer Science - Cryptography and Security_2017_8_2.json
144_Compute

It seems like they are not individual articles, so we have to look into one of them.
Let's just take the last one.

In [4]:
json_file = os.path.join(path, filename)
with open(json_file) as f:
    data = json.load(f)

JSON files are loaded as dictionaries:

In [5]:
data.keys()

dict_keys(['authors', 'contributors', 'datePublished', 'description', 'doi', 'downloadUrl', 'fullText', 'fulltextIdentifier', 'id', 'identifiers', 'oai', 'relations', 'repositories', 'subjects', 'title', 'topics', 'types', 'year'])

If we look into one of those keys, it seems like there is many articles, each one with their respective value for the keys above.

In [6]:
data['title'].keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'])

So we can check the content of the `Title` key, for all articles:

In [7]:
for key in data['title']:
    print(data['title'][key])
    print('-'*50)

Learning in the Machine: Random Backpropagation and the Deep Learning
  Channel
--------------------------------------------------
Imitation from Observation: Learning to Imitate Behaviors from Raw Video
  via Context Translation
--------------------------------------------------
Identifying hazardousness of sewer pipeline gas mixture using
  classification methods: a comparative study
--------------------------------------------------
Value Iteration Networks
--------------------------------------------------
Outrageously Large Neural Networks: The Sparsely-Gated
  Mixture-of-Experts Layer
--------------------------------------------------
A Review of Neural Network Based Machine Learning Approaches for Rotor
  Angle Stability Control
--------------------------------------------------
Learned Primal-dual Reconstruction
--------------------------------------------------
ACO for Continuous Function Optimization: A Performance Analysis
--------------------------------------------------
D

Or the content of all keys, for the first article, which is what we are looking for:

In [8]:
for key in data:
    print(key, data[key]['0'])
    print('-'*50)

authors ['Baldi, Pierre', 'Sadowski, Peter', 'Lu, Zhiqin']
--------------------------------------------------
contributors []
--------------------------------------------------
datePublished 2017-12-22
--------------------------------------------------
description Random backpropagation (RBP) is a variant of the backpropagation algorithm
for training neural networks, where the transpose of the forward matrices are
replaced by fixed random matrices in the calculation of the weight updates. It
is remarkable both because of its effectiveness, in spite of using random
matrices to communicate error information, and because it completely removes
the taxing requirement of maintaining symmetric weights in a physical neural
system. To better understand random backpropagation, we first connect it to the
notions of local learning and learning channels. Through this connection, we
derive several alternatives to RBP, including skipped RBP (SRPB), adaptive RBP
(ARBP), sparse RBP, and their combinati

Awesome! So we have to iterate for all the downloaded JSON files, for all keys, for all articles, so we can store it in a database. In this case we will use MongoDB.

## Extraction and Storage - MongoDB

We use MongoDB because it is great for prototyping and quick schema changes, but other storage systems can be probably used. Ultimately, all the data is then indexed in Elasticsearch, and that's where we actually query in production.

You can go [here](https://docs.mongodb.com/manual/installation/) to install MongoDB.

Then, using a MongoDB client, create a database, for example _pub_ and a collection, like _publications_.

In [2]:
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError
import string

In [3]:
# Default values
mongoDB_IP = '127.0.0.1'
mongoDB_Port = 27017
mongoDB_db = 'pub'

In [4]:
def connect_to_mongo():
    """
    Returns a db connection to the mongo instance
    :return:
    """
    try:
        client = MongoClient(mongoDB_IP, mongoDB_Port)
        db = client[mongoDB_db]
        db.downloads.find_one({'_id': 'test'})
        return db
    except ServerSelectionTimeoutError as e:
        raise Exception("Local MongoDB instance at "+mongoDB_IP+":"+mongoDB_Port+" could not be accessed.") from e

We can test the connection to the database with the following command:

In [5]:
db = connect_to_mongo()

Then we define a function that will take each of the articles from the JSON files downloaded from CORE, and store the information into our collection.

The data obtained from CORE does not separate sections in the text, it is not so important except for the references, since we may use them later on, therefore we also define a function that will manage this.

In [14]:
def arxiv_json_to_mongo(article):
    """
    Creates a new entry in mongodb from the input article
    :return:
    """
    
    mongo_input = {}
    translator = str.maketrans('', '', string.punctuation)
    article_data = article

    mongo_input['title'] = article_data['title']
    mongo_input['authors'] = article_data['authors']
    mongo_input['journal'] = 'arxiv'
    mongo_input['year'] = article_data['year']
    mongo_input['type'] = article_data['subjects']
    mongo_input['content.abstract'] = article_data['description']
    mongo_input['content.keywords'] = article_data['topics']
    mongo_input['content.fulltext'] = article_data['fullText']
    mongo_input['content.references'] = article_data['references']

    mongo_mongo_input = db.publications.update_one(
        {'_id': 'arxiv_' + article_data['id']},
        {'$set': mongo_input},
        upsert=True
    )    
    print('Processed', article_data['id'], 'from JSON')
    
def manage_text_and_refs(article):
    article['fullText'] = article['fullText'].replace('\n', ' ').replace('\r', '')
    if len(article['fullText'].split('References', 1)) == 2:
        article['references'] = article['fullText'].split('References', 1)[1].split('[')
        article['fullText'] = article['fullText'].split('References', 1)[0]
    elif len(article['fullText'].split('REFERENCES', 1)) == 2:
        article['references'] = article['fullText'].split('REFERENCES', 1)[1].split('[')
        article['fullText'] = article['fullText'].split('References', 1)[0]
    else:
        article['fullText'] = article['fullText']
        article['references'] = ''
    return article

Now we have to iterate through all the downloaded JSON files, and all the articles in each file, to process them and introduce them into our database.

In [16]:
for dirpath, dirnames, filenames in os.walk(path):
    # Iterate files...
    for filename in filenames:
        if filename.endswith('.json'):
            json_file = os.path.join(path, filename)
            with open(json_file) as f:
                data = json.load(f)            
                    # Iterate articles in file...
                for article_number in data['title'].keys():
                    article = {}
                    for key in data.keys():
                        article[key] = data[key][str(article_number)]
                        
                    # Process text and references for each article
                    manage_text_and_refs(article)
                    # Store to database
                    arxiv_json_to_mongo(article)

Processed 25036267 from JSON
Processed 2113087 from JSON
Processed 25025591 from JSON
Processed 25015102 from JSON
Processed 24956790 from JSON
Processed 25038409 from JSON
Processed 2246166 from JSON
Processed 73386897 from JSON
Processed 83863968 from JSON
Processed 83867371 from JSON
Processed 94047681 from JSON
Processed 146472595 from JSON
Processed 94063315 from JSON
Processed 29526388 from JSON
Processed 29536914 from JSON
Processed 84090473 from JSON
Processed 84093559 from JSON
Processed 84093915 from JSON
Processed 84093852 from JSON
Processed 84094420 from JSON
Processed 84326602 from JSON
Processed 84328172 from JSON
Processed 84330324 from JSON
Processed 84329889 from JSON
Processed 84326664 from JSON
Processed 84331310 from JSON
Processed 86416235 from JSON
Processed 86419366 from JSON
Processed 86420205 from JSON
Processed 84330603 from JSON
Processed 84330526 from JSON
Processed 84331497 from JSON
Processed 73423992 from JSON
Processed 73417539 from JSON
Processed 73408

Processed 93955298 from JSON
Processed 93957100 from JSON
Processed 93954637 from JSON
Processed 86415589 from JSON
Processed 86416049 from JSON
Processed 86421656 from JSON
Processed 86420019 from JSON
Processed 86416917 from JSON
Processed 84332112 from JSON
Processed 84331544 from JSON
Processed 86417035 from JSON
Processed 86415248 from JSON
Processed 86414384 from JSON
Processed 84331049 from JSON
Processed 129362579 from JSON
Processed 129353364 from JSON
Processed 129358112 from JSON
Processed 129363022 from JSON
Processed 129349022 from JSON
Processed 129348500 from JSON
Processed 129355601 from JSON
Processed 129355043 from JSON
Processed 129252805 from JSON
Processed 129263821 from JSON
Processed 129269449 from JSON
Processed 129276487 from JSON
Processed 94077449 from JSON
Processed 94026340 from JSON
Processed 73404689 from JSON
Processed 73402359 from JSON
Processed 73403592 from JSON
Processed 84090569 from JSON
Processed 84091942 from JSON
Processed 84092015 from JSON
Pr

Processed 9259374 from JSON
Processed 2418176 from JSON
Processed 24952807 from JSON
Processed 25041752 from JSON
Processed 25054883 from JSON
Processed 2064976 from JSON
Processed 25036302 from JSON
Processed 25015255 from JSON
Processed 25015890 from JSON
Processed 24978117 from JSON
Processed 24961009 from JSON
Processed 25063031 from JSON
Processed 25060134 from JSON
Processed 24960871 from JSON
Processed 25004536 from JSON
Processed 24975044 from JSON
Processed 25058543 from JSON
Processed 25015215 from JSON
Processed 2192099 from JSON
Processed 25050201 from JSON
Processed 25043984 from JSON
Processed 24947497 from JSON
Processed 25030700 from JSON
Processed 24960513 from JSON
Processed 2246166 from JSON
Processed 24965000 from JSON
Processed 146472672 from JSON
Processed 94066275 from JSON
Processed 84090954 from JSON
Processed 86419582 from JSON
Processed 86418022 from JSON
Processed 86420919 from JSON
Processed 84329360 from JSON
Processed 73407275 from JSON
Processed 73406878

Processed 73988612 from JSON
Processed 73988782 from JSON
Processed 73987842 from JSON
Processed 42701158 from JSON
Processed 73359320 from JSON
Processed 83837177 from JSON
Processed 83865527 from JSON
Processed 93955960 from JSON
Processed 94075803 from JSON
Processed 83847068 from JSON
Processed 93888370 from JSON
Processed 42739003 from JSON
Processed 93867540 from JSON
Processed 93875891 from JSON
Processed 129350143 from JSON
Processed 93874843 from JSON
Processed 83858768 from JSON
Processed 73349182 from JSON
Processed 73395899 from JSON
Processed 83857671 from JSON
Processed 83860794 from JSON
Processed 83845746 from JSON
Processed 73422623 from JSON
Processed 73400698 from JSON
Processed 73407079 from JSON
Processed 84093827 from JSON
Processed 29554804 from JSON
Processed 73409828 from JSON
Processed 84329847 from JSON
Processed 73954126 from JSON
Processed 73957459 from JSON
Processed 73989678 from JSON
Processed 42644498 from JSON
Processed 84328192 from JSON
Processed 742

Processed 141532918 from JSON
Processed 93897084 from JSON
Processed 83847336 from JSON
Processed 83859944 from JSON
Processed 84094692 from JSON
Processed 84093303 from JSON
Processed 83848897 from JSON
Processed 83848727 from JSON
Processed 83849218 from JSON
Processed 83852278 from JSON
Processed 83850272 from JSON
Processed 83842604 from JSON
Processed 73354579 from JSON
Processed 73377367 from JSON
Processed 83869728 from JSON
Processed 83863650 from JSON
Processed 83867397 from JSON
Processed 73351431 from JSON
Processed 129354096 from JSON
Processed 93946309 from JSON
Processed 93948590 from JSON
Processed 93953828 from JSON
Processed 93889927 from JSON
Processed 73423893 from JSON
Processed 29559494 from JSON
Processed 73406964 from JSON
Processed 73407518 from JSON
Processed 73416608 from JSON
Processed 29506180 from JSON
Processed 29530381 from JSON
Processed 73404261 from JSON
Processed 83856395 from JSON
Processed 73348729 from JSON
Processed 73381445 from JSON
Processed 73

...

Processed 25050344 from JSON
Processed 25056701 from JSON
Processed 25041752 from JSON
Processed 25048645 from JSON
Processed 24984510 from JSON
Processed 24938942 from JSON
Processed 24953401 from JSON
Processed 25013547 from JSON
Processed 24964557 from JSON
Processed 25015102 from JSON
Processed 24992766 from JSON
Processed 25029857 from JSON
Processed 2189676 from JSON
Processed 2184911 from JSON
Processed 25062049 from JSON
Processed 24956790 from JSON
Processed 25048018 from JSON
Processed 25023845 from JSON
Processed 25054674 from JSON
Processed 25030049 from JSON
Processed 25043675 from JSON
Processed 25034492 from JSON
Processed 24973903 from JSON
Processed 25030003 from JSON
Processed 25041490 from JSON
Processed 2246166 from JSON
Processed 24970122 from JSON
Processed 83864257 from JSON
Processed 83864163 from JSON
Processed 94063795 from JSON
Processed 84327713 from JSON
Processed 84328778 from JSON
Processed 86415573 from JSON
Processed 86416297 from JSON
Processed 8641952

Processed 78511424 from JSON
Processed 78510469 from JSON
Processed 78507670 from JSON
Processed 78508673 from JSON
Processed 78512100 from JSON
Processed 78508907 from JSON
Processed 129209900 from JSON
Processed 93903698 from JSON
Processed 94054926 from JSON
Processed 94065771 from JSON
Processed 93905687 from JSON
Processed 93911970 from JSON
Processed 93889189 from JSON
Processed 93943168 from JSON
Processed 93942879 from JSON
Processed 129355046 from JSON
Processed 129286933 from JSON
Processed 129349133 from JSON
Processed 129355231 from JSON
Processed 129353925 from JSON
Processed 93956902 from JSON
Processed 94026747 from JSON
Processed 141535174 from JSON
Processed 129240441 from JSON
Processed 129236628 from JSON
Processed 129251258 from JSON
Processed 129243806 from JSON
Processed 129269756 from JSON
Processed 129260672 from JSON
Processed 93946097 from JSON
Processed 93897029 from JSON
Processed 129359411 from JSON
Processed 83858300 from JSON
Processed 83852472 from JSON


Processed 24930779 from JSON
Processed 25030003 from JSON
Processed 86417517 from JSON
Processed 129205956 from JSON
Processed 94024259 from JSON
Processed 83860666 from JSON
Processed 129350991 from JSON
Processed 83857264 from JSON
Processed 83842145 from JSON
Processed 83838175 from JSON
Processed 73961361 from JSON
Processed 73421461 from JSON
Processed 42694258 from JSON
Processed 129241322 from JSON
Processed 141532259 from JSON
Processed 129356820 from JSON
Processed 83853876 from JSON
Processed 83849053 from JSON
Processed 86415541 from JSON
Processed 84330976 from JSON
Processed 129358390 from JSON
Processed 73421309 from JSON
Processed 84329069 from JSON
Processed 84327993 from JSON
Processed 78506856 from JSON
Processed 42745470 from JSON
Processed 83833862 from JSON
Processed 129359813 from JSON
Processed 129237940 from JSON
Processed 93940921 from JSON
Processed 129363302 from JSON
Processed 129362208 from JSON
Processed 83857623 from JSON
Processed 94050389 from JSON
Proc

Processed 93950900 from JSON
Processed 129286047 from JSON
Processed 83854092 from JSON
Processed 83844358 from JSON
Processed 84095274 from JSON
Processed 93955492 from JSON
Processed 93889881 from JSON
Processed 94022322 from JSON
Processed 141534671 from JSON
Processed 73992693 from JSON
Processed 42707453 from JSON
Processed 84333322 from JSON
Processed 84333214 from JSON
Processed 78508622 from JSON
Processed 141531711 from JSON
Processed 74202935 from JSON
Processed 83835579 from JSON
Processed 83835562 from JSON
Processed 83861303 from JSON
Processed 83839557 from JSON
Processed 93937297 from JSON
Processed 146476009 from JSON
Processed 94029742 from JSON
Processed 83848523 from JSON
Processed 83839293 from JSON
Processed 73358911 from JSON
Processed 93959051 from JSON
Processed 93950573 from JSON
Processed 93958762 from JSON
Processed 93942020 from JSON
Processed 42658277 from JSON
Processed 129361784 from JSON
Processed 84328381 from JSON
Processed 86415580 from JSON
Processed

Processed 84331949 from JSON
Processed 84326882 from JSON
Processed 42675139 from JSON
Processed 78507936 from JSON
Processed 78507176 from JSON
Processed 78510439 from JSON
Processed 42739776 from JSON
Processed 94044970 from JSON
Processed 83838920 from JSON
Processed 83839149 from JSON
Processed 83836948 from JSON
Processed 83860961 from JSON
Processed 83860822 from JSON
Processed 83860776 from JSON
Processed 83862378 from JSON
Processed 73381250 from JSON
Processed 73381591 from JSON
Processed 93913620 from JSON
Processed 93943418 from JSON
Processed 146473488 from JSON
Processed 146473363 from JSON
Processed 94067542 from JSON
Processed 94069454 from JSON
Processed 94074799 from JSON
Processed 129283260 from JSON
Processed 93873874 from JSON
Processed 93902293 from JSON
Processed 129227338 from JSON
Processed 93945609 from JSON
Processed 94066943 from JSON
Processed 94069144 from JSON
Processed 83847281 from JSON
Processed 83847654 from JSON
Processed 129210446 from JSON
Processed

Done! We have a bunch of files in our database! (**3814** articles in this demo)

## Indexing - Elasticsearch 

We use Elasticsearch for the quick and very flexible (elastic?) queries across the full text of articles, so we have to index all the content from the database for this.

Once again, we have to [install](https://www.elastic.co/guide/en/elasticsearch/reference/current/_installation.html) it and run the service...yay!

Once it's running we can connect and check status:

In [6]:
import pymongo
import elasticsearch
import requests
import nltk
from elasticsearch import helpers
import re
import string

In [7]:
client = pymongo.MongoClient('localhost:' + str(mongoDB_Port))
publications_collection = client.pub.publications
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)

In [26]:
es.cluster.health()

{'active_primary_shards': 0,
 'active_shards': 0,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

### Full text indexing

First we index the full content of every article in the database.

(This step corresponds to `collection_extraction -> index_papersfulltext.py` in the code)

In [14]:
def extract_metadata(documents):
    list_of_docs = []
    for i, r in enumerate(documents):
        extracted = {
                "_id": "",
                "title": "",
                "publication": "",
                "year": "",
                "content": "",
                "abstract": "",
                "authors": [],
                "references": []}
        extracted['_id'] = r['_id']
        extracted['title'] = r['title']
        extracted['publication'] = r['journal']
        extracted['year'] = r['year']
        extracted['content'] = r['content']['fulltext']
        extracted['abstract'] = r['content']['abstract']
        extracted['authors'] = r['authors']
        extracted['references'] = r['content']['references']
        list_of_docs.append(extracted)
    return list_of_docs

In [15]:
filter_publications = ['arxiv'] # Here we could also put PubMed or other sources

extracted_publications = []
for publication in filter_publications:
    query ={'$and': [{'journal': publication}, {'content.fulltext': {'$exists': True}}]}                   
    results = publications_collection.find(query)
    extracted_publications.append(extract_metadata(results))

In [29]:
for publication in extracted_publications:
    actions = []
    for article in publication:
        authors = []
        if len(article['authors']) > 0:
            if type(article['authors'][0]) == list:
                try:
                    for name in article['authors']:
                        authors.append(', '.join([name[1], name[0]]))
                    authors = list(set(authors))
                except:
                    pass
            else:
                authors = article['authors']
        actions.append({
            "_index": "ir", 
            "_type": "publications",  
            "_id": article['_id'],
            "_source": {
                "title": article["title"],
                "journal": article['publication'],
                "year": str(article['year']),
                "content": article["content"],
                "abstract": article["abstract"],
                "authors": authors,
                "references": article["references"]
            }
        })
    if len(actions) == 0:
        continue
    res = helpers.bulk(es, actions)
    print('Done with', res, 'articles added to index')

Done with (3814, []) articles added to index


We can look for anything in the text, like this:

In [30]:
res = es.search(index = "ir", body = {"query": {"match": {"title" : "netflix"}}}, size = 10)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print(hit['_id'], hit['_source']['abstract'])
    print('-'*50)

Got 1 Hits:
arxiv_84094279 Re-Evaluating the Netflix Prize - Human Uncertainty and its Impact on
  Reliability
arxiv_84094279 In this paper, we examine the statistical soundness of comparative
assessments within the field of recommender systems in terms of reliability and
human uncertainty. From a controlled experiment, we get the insight that users
provide different ratings on same items when repeatedly asked. This volatility
of user ratings justifies the assumption of using probability densities instead
of single rating scores. As a consequence, the well-known accuracy metrics
(e.g. MAE, MSE, RMSE) yield a density themselves that emerges from convolution
of all rating densities. When two different systems produce different RMSE
distributions with significant intersection, then there exists a probability of
error for each possible ranking. As an application, we examine possible ranking
errors of the Netflix Prize. We are able to show that all top rankings are more
or less subject to h

### Sentence Indexing 
For the sentence expansion step of our process, we need to have sentences indexed. This is because we need to find similar sentences as well as their surrounding sentences, for context. 

In addition, we create a file with all the text, of all our articles, and we will use it later for the training of embedding models.

(This step corresponds to `collection_extraction -> index_twosent.py` in the code)

In [8]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [17]:
for publication in extracted_publications:
    for article in publication:
        actions = []
        cleaned = []
        dataset_sent = []
        other_sent = []

        lines = (sent_detector.tokenize(article['content'].strip()))
        
        # This will be used for the training of word2vec and doc2vec
        # You need to create the data folder beforehand
        with open('data/full_text_corpus.txt', 'a', encoding='utf-8') as f:
            for line in lines:
                f.write(line)
        f.close()
        
        if len(lines) < 3:
            continue

        for i in range(len(lines)):
            words = nltk.word_tokenize(lines[i])
            word_lengths = [len(x) for x in words]
            average_word_length = sum(word_lengths) / len(word_lengths)
            if average_word_length < 4:
                continue

            two_sentences = ''
            try:
                two_sentences = lines[i] + ' ' + lines[i - 1]
            except:
                two_sentences = lines[i] + ' ' + lines[i + 1]

            dataset_sent.append(two_sentences)

        for num, added_lines in enumerate(dataset_sent):
            actions.append({
                "_index": "twosent",
                "_type": "twosentnorules",
                "_id": article['_id'] + str(num),
                "_source": {
                    "title": article['title'],
                    "content.chapter.sentpositive": added_lines,
                    "paper_id": article['_id']
                }})

        if len(actions) == 0:
            continue
        res = helpers.bulk(es, actions)
print('Done')

Done


In [9]:
res = es.search(index = "twosent", body = {"query": {"match": {"content.chapter.sentpositive" : "imagenet"}}}, size = 5)

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit['_id'], hit['_source']['title'])
    print(hit['_source']['content.chapter.sentpositive'])
    print('-'*50)

Got 1311 Hits:
arxiv_94074613218 Expert Gate: Lifelong Learning with a Network of Experts
We consider the following knowl- edge transfer cases: Scenes → Actions, ImageNet → Ac- tions, SVHN → Letters, ImageNet → Letters, SVHN → Mnist, ImageNet→ Mnist, Flowers→ Cars and Imagenet → Cars. We also consider ImageNet as a possible source.
--------------------------------------------------
arxiv_93937583198 Learning Efficient Convolutional Networks through Network Slimming
The results for ImageNet dataset are summa- rized in Table 2. ImageNet.
--------------------------------------------------
arxiv_86420809239 ZOO: Zeroth Order Optimization based Black-box Attacks to Deep Neural
  Networks without Training Substitute Models
Substi- tute model based attack cannot easily scale to ImageNet. Table 2: Untargeted ImageNet attacks comparison.
--------------------------------------------------
arxiv_93937583152 Learning Efficient Convolutional Networks through Network Slimming
The ImageNet dataset co

### Doc2vec Indexing

Once again, we index the sentences from the full text file created before, which will be used for the Sentence Expansion using doc2vec.

(This step corresponds to `collection_extraction -> index_doc2vec.py` in the code)

In [10]:
file = open('data/full_text_corpus.txt', 'r', encoding='utf-8')
text = file.read()
file.close()
sentences = nltk.tokenize.sent_tokenize(text)
print('Sentences ready')
count = 0
docLabels = []
actions = []

for i, sent in enumerate(sentences):
    try:
        neighbors = sentences[i + 1]
        neighbor_count = count + 1
    except:
        neighbors = sentences[i - 1]
        neighbor_count = count - 1

    docLabels.append(count)
    actions.append({
        "_index": "devtwosentnew",
        "_type": "devtwosentnorulesnew",
        "_id": count,
        "_source": {
            "content.chapter.sentpositive": sent,
            "content.chapter.sentnegtive": neighbors,
            "neighborcount": neighbor_count
        }})
    count = count + 1

print(len(sentences))
print(len(docLabels))
res = helpers.bulk(es, actions)
print(res)

Sentences ready
112410
112410
(112410, [])


## Training Embedding Models (word2vec & doc2vec)

For the Term and Sentence expansion, we need to find similar words or sentences, and we use word2vec and doc2vec respectively. We have to generate a document with all the sentences in our corpus, which will be stored in the data folder. While this is not a full implementation, we have around 4 million sentences, so it is a considerable example, and processing can take some time.

(This step corresponds to `data_preparation -> prepare_embedding_data.py` in the code)

In [16]:
sentence_text = []
translator = str.maketrans('', '', string.punctuation)

for publication in extracted_publications:
    for article in publication:
        query = {"query":
                     {"match":
                          {"_id":
                               {"query": article['_id'],
                                "operator": "and"
                                }
                           }
                      }
                 }

        results = es.search(index="ir", body=query, size=200)
        for doc in results['hits']['hits']:
            fulltext = doc["_source"]["content"]
            fulltext = re.sub("[\[].*?[\]]", "", fulltext)
            sentence_text.append(fulltext)
    print('Done', '-' * 100)

f = open("data/data2vec.txt", "w", encoding='utf-8')
for line in sentence_text:
    f.write(line)
f.close()

Done ----------------------------------------------------------------------------------------------------


Before training, verify that you have the optimized version of Gensim using Cython, for better performance.

In [18]:
import gensim
assert gensim.models.doc2vec.FAST_VERSION > -1

Now that we have the document with all the text, we can use them for the training. 

We will use a bigram model for word2vec, since it's common that entities are represented as two words. You can run the command from your Python console or directly here, using the magic `%run` syntax. The structure of this command is as follows:

`%run script_to_run training_data model_output word_vector_output`

This was run in a normal Lenovo machine with 8GB and i7 processor. As you can see by the timestamps below, it might take a while...

In [19]:
%run data_preparation/train_word2vec.py data/data2vec.txt embedding_models/modelword2vecbigram.model embedding_models/modelword2vecbigram.vec

2018-05-15 17:07:25,101: INFO: running data_preparation/train_word2vec.py data/data2vec.txt embedding_models/modelword2vecbigram.model embedding_models/modelword2vecbigram.vec
2018-05-15 17:17:08,824: INFO: collecting all words and their counts
2018-05-15 17:17:08,831: INFO: PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-05-15 17:17:09,350: INFO: PROGRESS: at sentence #10000, processed 201700 words and 71739 word types
2018-05-15 17:17:09,858: INFO: PROGRESS: at sentence #20000, processed 398393 words and 134942 word types
2018-05-15 17:17:10,411: INFO: PROGRESS: at sentence #30000, processed 617098 words and 204286 word types
2018-05-15 17:17:11,162: INFO: PROGRESS: at sentence #40000, processed 862142 words and 279857 word types
2018-05-15 17:17:11,845: INFO: PROGRESS: at sentence #50000, processed 1119381 words and 356880 word types
2018-05-15 17:17:12,539: INFO: PROGRESS: at sentence #60000, processed 1357143 words and 420874 word types
2018-05-15 17:17:13,233: I

2018-05-15 17:17:55,380: INFO: PROGRESS: at sentence #740000, processed 16120117 words and 2597213 word types
2018-05-15 17:17:56,083: INFO: PROGRESS: at sentence #750000, processed 16345603 words and 2637256 word types
2018-05-15 17:17:56,753: INFO: PROGRESS: at sentence #760000, processed 16571765 words and 2675889 word types
2018-05-15 17:17:57,412: INFO: PROGRESS: at sentence #770000, processed 16792631 words and 2716795 word types
2018-05-15 17:17:57,987: INFO: PROGRESS: at sentence #780000, processed 16981822 words and 2749520 word types
2018-05-15 17:17:58,700: INFO: PROGRESS: at sentence #790000, processed 17225297 words and 2783581 word types
2018-05-15 17:17:59,690: INFO: PROGRESS: at sentence #800000, processed 17459556 words and 2822675 word types
2018-05-15 17:18:00,351: INFO: PROGRESS: at sentence #810000, processed 17683519 words and 2858184 word types
2018-05-15 17:18:01,101: INFO: PROGRESS: at sentence #820000, processed 17920868 words and 2895177 word types
2018-05-15

2018-05-15 17:18:47,773: INFO: PROGRESS: at sentence #1490000, processed 33089396 words and 4905102 word types
2018-05-15 17:18:48,349: INFO: PROGRESS: at sentence #1500000, processed 33316021 words and 4928569 word types
2018-05-15 17:18:48,983: INFO: PROGRESS: at sentence #1510000, processed 33538115 words and 4946507 word types
2018-05-15 17:18:49,781: INFO: PROGRESS: at sentence #1520000, processed 33811072 words and 4965999 word types
2018-05-15 17:18:50,274: INFO: PROGRESS: at sentence #1530000, processed 33983474 words and 4994608 word types
2018-05-15 17:18:50,843: INFO: PROGRESS: at sentence #1540000, processed 34189105 words and 5026079 word types
2018-05-15 17:18:51,448: INFO: PROGRESS: at sentence #1550000, processed 34433445 words and 5059287 word types
2018-05-15 17:18:51,977: INFO: PROGRESS: at sentence #1560000, processed 34644834 words and 5091811 word types
2018-05-15 17:18:52,538: INFO: PROGRESS: at sentence #1570000, processed 34866405 words and 5131430 word types
2

2018-05-15 17:21:09,980: INFO: PROGRESS: at sentence #230000, processed 4069387 words, keeping 256004 word types
2018-05-15 17:21:10,738: INFO: PROGRESS: at sentence #240000, processed 4214508 words, keeping 261233 word types
2018-05-15 17:21:11,743: INFO: PROGRESS: at sentence #250000, processed 4410567 words, keeping 268577 word types
2018-05-15 17:21:12,814: INFO: PROGRESS: at sentence #260000, processed 4617727 words, keeping 275896 word types
2018-05-15 17:21:13,804: INFO: PROGRESS: at sentence #270000, processed 4808859 words, keeping 283390 word types
2018-05-15 17:21:14,787: INFO: PROGRESS: at sentence #280000, processed 5001159 words, keeping 290583 word types
2018-05-15 17:21:15,680: INFO: PROGRESS: at sentence #290000, processed 5179180 words, keeping 295423 word types
2018-05-15 17:21:16,542: INFO: PROGRESS: at sentence #300000, processed 5346906 words, keeping 301698 word types
2018-05-15 17:21:17,447: INFO: PROGRESS: at sentence #310000, processed 5521343 words, keeping 3

2018-05-15 17:22:20,257: INFO: PROGRESS: at sentence #960000, processed 17045165 words, keeping 622723 word types
2018-05-15 17:22:21,063: INFO: PROGRESS: at sentence #970000, processed 17206815 words, keeping 626420 word types
2018-05-15 17:22:21,860: INFO: PROGRESS: at sentence #980000, processed 17366391 words, keeping 631957 word types
2018-05-15 17:22:22,792: INFO: PROGRESS: at sentence #990000, processed 17552913 words, keeping 640484 word types
2018-05-15 17:22:23,665: INFO: PROGRESS: at sentence #1000000, processed 17723623 words, keeping 647237 word types
2018-05-15 17:22:24,577: INFO: PROGRESS: at sentence #1010000, processed 17884475 words, keeping 653943 word types
2018-05-15 17:22:25,382: INFO: PROGRESS: at sentence #1020000, processed 18045564 words, keeping 659946 word types
2018-05-15 17:22:26,194: INFO: PROGRESS: at sentence #1030000, processed 18204849 words, keeping 665730 word types
2018-05-15 17:22:27,043: INFO: PROGRESS: at sentence #1040000, processed 18374412 wo

...

2018-05-15 17:38:40,202: INFO: EPOCH 4 - PROGRESS: at 72.15% examples, 106787 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:38:41,306: INFO: EPOCH 4 - PROGRESS: at 72.45% examples, 106601 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:38:42,460: INFO: EPOCH 4 - PROGRESS: at 72.65% examples, 106209 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:38:43,568: INFO: EPOCH 4 - PROGRESS: at 72.90% examples, 105929 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:38:44,608: INFO: EPOCH 4 - PROGRESS: at 73.25% examples, 105792 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:38:45,634: INFO: EPOCH 4 - PROGRESS: at 73.58% examples, 105666 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:38:46,635: INFO: EPOCH 4 - PROGRESS: at 73.90% examples, 105598 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:38:47,651: INFO: EPOCH 4 - PROGRESS: at 74.31% examples, 105560 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:38:48,702: INFO: EPOCH 4 - PROGRESS: at 74.71% examples, 105419 words/s, in_qsize 0, out_qsize 0
2

2018-05-15 17:39:58,055: INFO: EPOCH 4 - PROGRESS: at 96.85% examples, 98046 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:39:59,158: INFO: EPOCH 4 - PROGRESS: at 97.11% examples, 97874 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:00,226: INFO: EPOCH 4 - PROGRESS: at 97.40% examples, 97748 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:01,288: INFO: EPOCH 4 - PROGRESS: at 97.77% examples, 97621 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:02,397: INFO: EPOCH 4 - PROGRESS: at 98.01% examples, 97479 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:03,492: INFO: EPOCH 4 - PROGRESS: at 98.28% examples, 97344 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:04,541: INFO: EPOCH 4 - PROGRESS: at 98.55% examples, 97196 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:05,671: INFO: EPOCH 4 - PROGRESS: at 98.81% examples, 97019 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:40:06,743: INFO: EPOCH 4 - PROGRESS: at 99.08% examples, 96895 words/s, in_qsize 0, out_qsize 0
2018-05-15

2018-05-15 17:41:10,799: INFO: EPOCH 5 - PROGRESS: at 18.55% examples, 74901 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:41:11,828: INFO: EPOCH 5 - PROGRESS: at 18.92% examples, 75293 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:41:12,866: INFO: EPOCH 5 - PROGRESS: at 19.37% examples, 75627 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:41:13,899: INFO: EPOCH 5 - PROGRESS: at 19.79% examples, 75872 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:41:14,964: INFO: EPOCH 5 - PROGRESS: at 20.20% examples, 76055 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:41:15,989: INFO: EPOCH 5 - PROGRESS: at 20.56% examples, 76292 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:41:16,994: INFO: EPOCH 5 - PROGRESS: at 20.99% examples, 76647 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:41:17,996: INFO: EPOCH 5 - PROGRESS: at 21.36% examples, 76902 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:41:19,094: INFO: EPOCH 5 - PROGRESS: at 21.77% examples, 77146 words/s, in_qsize 0, out_qsize 0
2018-05-15

2018-05-15 17:42:28,969: INFO: EPOCH 5 - PROGRESS: at 49.23% examples, 86748 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:42:29,996: INFO: EPOCH 5 - PROGRESS: at 49.61% examples, 86736 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:42:31,032: INFO: EPOCH 5 - PROGRESS: at 49.93% examples, 86668 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:42:32,068: INFO: EPOCH 5 - PROGRESS: at 50.22% examples, 86553 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:42:33,092: INFO: EPOCH 5 - PROGRESS: at 50.59% examples, 86654 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:42:34,130: INFO: EPOCH 5 - PROGRESS: at 50.99% examples, 86750 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:42:35,175: INFO: EPOCH 5 - PROGRESS: at 51.54% examples, 86881 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:42:36,237: INFO: EPOCH 5 - PROGRESS: at 52.01% examples, 87108 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:42:37,305: INFO: EPOCH 5 - PROGRESS: at 52.51% examples, 87273 words/s, in_qsize 0, out_qsize 0
2018-05-15

2018-05-15 17:43:46,906: INFO: EPOCH 5 - PROGRESS: at 80.04% examples, 91419 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:47,965: INFO: EPOCH 5 - PROGRESS: at 80.63% examples, 91538 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:49,037: INFO: EPOCH 5 - PROGRESS: at 81.16% examples, 91655 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:50,078: INFO: EPOCH 5 - PROGRESS: at 81.57% examples, 91715 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:51,241: INFO: EPOCH 5 - PROGRESS: at 81.92% examples, 91627 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:52,307: INFO: EPOCH 5 - PROGRESS: at 82.35% examples, 91680 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:53,373: INFO: EPOCH 5 - PROGRESS: at 82.79% examples, 91737 words/s, in_qsize 0, out_qsize 0
2018-05-15 17:43:54,429: INFO: EPOCH 5 - PROGRESS: at 83.22% examples, 91854 words/s, in_qsize 1, out_qsize 0
2018-05-15 17:43:55,454: INFO: EPOCH 5 - PROGRESS: at 83.66% examples, 91893 words/s, in_qsize 0, out_qsize 0
2018-05-15

Now we select the sentences from the full text and use them for the doc2vec training. Once again, we can run the command from your Python console or directly here, using the magic `%run` syntax. The structure of this command is as follows:

`%run script_to_run training_data model_output`

This also took some time.

In [55]:
%run data_preparation/train_doc2vec.py data/data2vec.txt embedding_models/doc2vec.model

2018-05-14 23:23:49,487: INFO: running data_preparation/train_doc2vec.py data/dataDoc2vec.txt data/doc2vec.model
2018-05-14 23:27:02,732: INFO: collecting all words and their counts
C:\Users\mvall\PycharmProjects\SmartPub-TSENER\data_preparation\train_doc2vec.py:45: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  yield LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])
2018-05-14 23:27:02,757: INFO: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-14 23:27:02,873: INFO: PROGRESS: at example #10000, processed 192995 words (1691505/s), 27656 word types, 10000 tags


4138892
4138892


2018-05-14 23:27:02,992: INFO: PROGRESS: at example #20000, processed 377367 words (1559424/s), 43708 word types, 20000 tags
2018-05-14 23:27:03,087: INFO: PROGRESS: at example #30000, processed 508001 words (1388833/s), 52207 word types, 30000 tags
2018-05-14 23:27:03,193: INFO: PROGRESS: at example #40000, processed 686456 words (1700594/s), 65978 word types, 40000 tags
2018-05-14 23:27:03,306: INFO: PROGRESS: at example #50000, processed 880869 words (1734820/s), 79708 word types, 50000 tags
2018-05-14 23:27:03,428: INFO: PROGRESS: at example #60000, processed 1092356 words (1766510/s), 93801 word types, 60000 tags
2018-05-14 23:27:03,547: INFO: PROGRESS: at example #70000, processed 1284802 words (1626946/s), 105557 word types, 70000 tags
2018-05-14 23:27:03,658: INFO: PROGRESS: at example #80000, processed 1476281 words (1734553/s), 116196 word types, 80000 tags
2018-05-14 23:27:03,770: INFO: PROGRESS: at example #90000, processed 1663771 words (1692946/s), 127655 word types, 9000

2018-05-14 23:27:10,404: INFO: PROGRESS: at example #660000, processed 12732835 words (1664873/s), 526757 word types, 660000 tags
2018-05-14 23:27:10,524: INFO: PROGRESS: at example #670000, processed 12949445 words (1829706/s), 533368 word types, 670000 tags
2018-05-14 23:27:10,636: INFO: PROGRESS: at example #680000, processed 13140942 words (1719813/s), 539006 word types, 680000 tags
2018-05-14 23:27:10,755: INFO: PROGRESS: at example #690000, processed 13351149 words (1786536/s), 543682 word types, 690000 tags
2018-05-14 23:27:10,879: INFO: PROGRESS: at example #700000, processed 13551907 words (1626434/s), 549806 word types, 700000 tags
2018-05-14 23:27:11,007: INFO: PROGRESS: at example #710000, processed 13763406 words (1677777/s), 557006 word types, 710000 tags
2018-05-14 23:27:11,125: INFO: PROGRESS: at example #720000, processed 13960015 words (1683663/s), 562340 word types, 720000 tags
2018-05-14 23:27:11,244: INFO: PROGRESS: at example #730000, processed 14153815 words (163

2018-05-14 23:27:17,613: INFO: PROGRESS: at example #1290000, processed 24615959 words (1580936/s), 847614 word types, 1290000 tags
2018-05-14 23:27:17,718: INFO: PROGRESS: at example #1300000, processed 24784655 words (1630871/s), 853677 word types, 1300000 tags
2018-05-14 23:27:17,838: INFO: PROGRESS: at example #1310000, processed 24981022 words (1649469/s), 860911 word types, 1310000 tags
2018-05-14 23:27:17,937: INFO: PROGRESS: at example #1320000, processed 25130850 words (1523873/s), 866810 word types, 1320000 tags
2018-05-14 23:27:18,042: INFO: PROGRESS: at example #1330000, processed 25298357 words (1621912/s), 872001 word types, 1330000 tags
2018-05-14 23:27:18,146: INFO: PROGRESS: at example #1340000, processed 25466785 words (1636203/s), 876442 word types, 1340000 tags
2018-05-14 23:27:18,251: INFO: PROGRESS: at example #1350000, processed 25636602 words (1626704/s), 880499 word types, 1350000 tags
2018-05-14 23:27:18,363: INFO: PROGRESS: at example #1360000, processed 2580

...

2018-05-15 01:06:35,978: INFO: EPOCH 4 - PROGRESS: at 90.43% examples, 42201 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:36,992: INFO: EPOCH 4 - PROGRESS: at 90.76% examples, 42349 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:38,015: INFO: EPOCH 4 - PROGRESS: at 91.11% examples, 42498 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:39,020: INFO: EPOCH 4 - PROGRESS: at 91.48% examples, 42660 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:40,024: INFO: EPOCH 4 - PROGRESS: at 91.81% examples, 42801 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:41,050: INFO: EPOCH 4 - PROGRESS: at 92.18% examples, 42948 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:42,135: INFO: EPOCH 4 - PROGRESS: at 92.55% examples, 43091 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:06:43,148: INFO: EPOCH 4 - PROGRESS: at 92.93% examples, 43239 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:06:44,162: INFO: EPOCH 4 - PROGRESS: at 93.30% examples, 43385 words/s, in_qsize 16, out_qsize 0
2

2018-05-15 01:13:03,539: INFO: EPOCH 5 - PROGRESS: at 13.87% examples, 26096 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:04,553: INFO: EPOCH 5 - PROGRESS: at 14.17% examples, 26630 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:05,609: INFO: EPOCH 5 - PROGRESS: at 14.48% examples, 27094 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:06,644: INFO: EPOCH 5 - PROGRESS: at 14.86% examples, 27698 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:07,660: INFO: EPOCH 5 - PROGRESS: at 15.19% examples, 28299 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:08,669: INFO: EPOCH 5 - PROGRESS: at 15.58% examples, 28843 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:09,768: INFO: EPOCH 5 - PROGRESS: at 15.96% examples, 29495 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:10,800: INFO: EPOCH 5 - PROGRESS: at 16.33% examples, 30145 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:13:11,828: INFO: EPOCH 5 - PROGRESS: at 16.68% examples, 30729 words/s, in_qsize 16, out_qsize 0
2

2018-05-15 01:19:42,154: INFO: EPOCH 5 - PROGRESS: at 40.57% examples, 34921 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:19:43,173: INFO: EPOCH 5 - PROGRESS: at 40.91% examples, 35144 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:19:44,178: INFO: EPOCH 5 - PROGRESS: at 41.30% examples, 35388 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:19:45,213: INFO: EPOCH 5 - PROGRESS: at 41.69% examples, 35643 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:19:46,264: INFO: EPOCH 5 - PROGRESS: at 42.06% examples, 35906 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:19:47,374: INFO: EPOCH 5 - PROGRESS: at 42.49% examples, 36163 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:19:48,394: INFO: EPOCH 5 - PROGRESS: at 42.88% examples, 36402 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:19:49,415: INFO: EPOCH 5 - PROGRESS: at 43.22% examples, 36640 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:19:50,456: INFO: EPOCH 5 - PROGRESS: at 43.59% examples, 36876 words/s, in_qsize 15, out_qsize 0
2

2018-05-15 01:26:27,208: INFO: EPOCH 5 - PROGRESS: at 66.82% examples, 37191 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:28,209: INFO: EPOCH 5 - PROGRESS: at 67.20% examples, 37368 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:29,217: INFO: EPOCH 5 - PROGRESS: at 67.53% examples, 37531 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:30,242: INFO: EPOCH 5 - PROGRESS: at 67.92% examples, 37677 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:31,264: INFO: EPOCH 5 - PROGRESS: at 68.28% examples, 37855 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:32,348: INFO: EPOCH 5 - PROGRESS: at 68.64% examples, 38028 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:33,349: INFO: EPOCH 5 - PROGRESS: at 69.04% examples, 38198 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:34,374: INFO: EPOCH 5 - PROGRESS: at 69.40% examples, 38368 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:26:35,396: INFO: EPOCH 5 - PROGRESS: at 69.74% examples, 38534 words/s, in_qsize 15, out_qsize 0
2

2018-05-15 01:33:08,552: INFO: EPOCH 5 - PROGRESS: at 92.36% examples, 38212 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:33:09,637: INFO: EPOCH 5 - PROGRESS: at 92.56% examples, 38273 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:33:10,695: INFO: EPOCH 5 - PROGRESS: at 92.90% examples, 38390 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:33:11,714: INFO: EPOCH 5 - PROGRESS: at 93.25% examples, 38512 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:33:12,715: INFO: EPOCH 5 - PROGRESS: at 93.62% examples, 38646 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:33:13,719: INFO: EPOCH 5 - PROGRESS: at 93.95% examples, 38773 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:33:14,740: INFO: EPOCH 5 - PROGRESS: at 94.28% examples, 38901 words/s, in_qsize 16, out_qsize 0
2018-05-15 01:33:15,753: INFO: EPOCH 5 - PROGRESS: at 94.65% examples, 39038 words/s, in_qsize 15, out_qsize 0
2018-05-15 01:33:16,855: INFO: EPOCH 5 - PROGRESS: at 95.03% examples, 39169 words/s, in_qsize 15, out_qsize 0
2

And this is it for data collection, extraction and preparation!

The next step is the generation of training data (with Term and Sentence Expansion) for the Named-Entity Tagger and, of course, the actual training.